### Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U colabcode
import colabcode
colabcode.ColabCode(
    port=12000,
    authtoken='21eNVw6XbU2fjExgUWxa47wNx8x_6AfyfxMFYUmSWFLxW7b72',
    lab=True
)

Mounted at /content/drive
     |████████████████████████████████| 745 kB 5.1 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 8.3 MB 38.6 MB/s 
     |████████████████████████████████| 428 kB 33.0 MB/s 
     |████████████████████████████████| 396 kB 58.6 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 129 kB 73.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 125 kB 74.7 MB/s 
     |████████████████████████████████| 124 kB 71.0 MB/s 
     |████████████████████████████████| 124 kB 56.2 MB/s 
     |████████████████████████████████| 122 kB 55.2 MB/s 
     |████████████████████████████████| 122 kB 47.3 MB/s 
     |████████████████████████████████| 122 kB 69.8 MB/s 
     |████████████████████████████████| 122 kB 59.4 MB/s 

Code Server can be accessed on: NgrokTunnel: "https://d198-35-190-183-137.ngrok.io" -> "http://localhost:12000"
Jupyter lab token: 02a828e0-5806-11ec-896c-0242ac1c0002


In [1]:
!gdown --id 1A-59GMNuJWbrbCc17zvBPkOycSdCBdCA

Downloading...
From: https://drive.google.com/uc?id=1A-59GMNuJWbrbCc17zvBPkOycSdCBdCA
To: /content/vacancies_lemmatized.csv
100% 902M/902M [00:05<00:00, 164MB/s]  


In [21]:
import pandas as pd
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']

### Лемматизация

In [ ]:
!gdown --id 1YsX2nb2IxiKICa7S1b5LLMNC0puVwwry

In [ ]:
df = pd.read_csv('vacancies_processed.csv')

In [ ]:
# !pip install pymorphy2
import pymorphy2
import re
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
  """
  Создаём метод, который лемматизирует текст.
  Аргумент:
  text - текст, который нужно лемматизировать
  Возвращает:
  res - лемматизированный текст
  """
  try:
      words = re.split(r'\W+', text) # разбиваем текст на слова
  except:
    return text
  res = str()
  for word in words:
      p = morph.parse(word)[0]
      res += (p.normal_form) + ' '
  return res

In [ ]:
columns_for_lem = ['raw_description', 'description', 'description_responsibilities', 'description_requirements', 'description_conditions']
lemmatized_columns = ['lem_raw_desc', 'lem_desc', 'lem_desc_resp', 'lem_desc_req', 'lem_desc_con']
for j in range(len(columns_for_lem)):
  df[lemmatized_columns[j]] = 0
  df[lemmatized_columns[j]][df['language'] == 'ru'] = df[columns_for_lem[j]][df['language'] == 'ru'].apply(lemmatize)

In [ ]:
df.to_csv('vacancies_lemmatized.csv', index=False)

### Модели

In [22]:
# классифицируем prof_area, фичи - tfidf над столбцом lem_raw_desc (max_features=4000)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from tabulate import tabulate
df = df[['lem_raw_desc', 'specialization', 'specializations_name', 'len_description_responsibilities', 'len_description_requirements', 'len_description_conditions']][df['language'] == 'ru'].dropna()

In [23]:
def fill_prof_area(prof_areas, row:  list):
  """
  Создаём метод, который заполняет словарь prof_area сферами и частотой их встречания в вакансиях
  Аргумент:
  row - строка в датафрейме
  """
  for el in row:
    if el in prof_areas:
      prof_areas[el] += 1
    else:
      prof_areas[el] = 1

def decrease_specializations(x: str):
  """
  Создаём метод, которые вместо наименее популярных специализаций пишет "Другое"
  Аргумент:
  х - специализация
  Возвращает:
  х - специализация
  """
  try:
    if specializations[x]/len(df) < 0.01:
      return 'Другое'
    else:
      return x
  except:
    return x

specializations = dict()
df['specializations_name'].apply(lambda x: fill_prof_area(specializations, eval(x)));

In [24]:
df['specialization'] = df['specialization'].apply(decrease_specializations)

In [25]:
X = df[['lem_raw_desc', 'specialization']].dropna()
X_train, X_test, y_train, y_test = train_test_split(X['lem_raw_desc'], X['specialization'], test_size=0.2, random_state=42)

In [26]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=3500)

# We transform each complaint into a vector

tfidf.fit(X_train)
features = tfidf.transform(X_train)

In [27]:
# Добавляем фичи с длинами массивов
feat = ['len_description_responsibilities', 'len_description_requirements', 'len_description_conditions']
features = pd.DataFrame(features.toarray())
for feature in feat:
  features[feat] = df[feat]

In [ ]:
for col in X_train.columns:
  if col != 'description':
    features = pd.concat([features,
                          pd.DataFrame(tfidf.fit_transform(X_train[col].values.astype('U')).toarray())], axis=1)

In [28]:
del X_train

In [ ]:
model = RandomForestClassifier()
model.fit(features, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
del features

In [11]:
features_test = pd.DataFrame(tfidf.transform(X_test.values.astype('U')).toarray())

In [ ]:
for col in X_test.columns:
  if col != 'description':
    features_test = pd.concat([features_test,
                          pd.DataFrame(tfidf.transform(X_test[col].values.astype('U')).toarray())], axis=1)

In [12]:
y_pred = model.predict(features_test)

In [13]:
# Сохраняем модель
import joblib
joblib.dump(model, 'lem_desc_tfidf_RF_specialization.pkl', compress=8)

['lem_desc_tfidf_RF_specialization.pkl']

In [17]:
salary_groupped_prof_area = df[['specialization', 'salary_from']].groupby(['specialization']).mean()
salary_pred = salary_groupped_prof_area.loc[y_pred]
salary_test = df.loc[y_test.index, 'salary_from']

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
salary = pd.DataFrame()
salary['y_pred'] = y_pred
salary['test'] =  list(salary_test)
salary['pred'] = list(salary_pred['salary_from'])
salary['true_prof_area'] = list(df.loc[y_test.index, 'specialization'])
salary = salary.dropna()
regression_report = [['specialization', 'MAE']]
for area in salary['true_prof_area'].unique():
  regression_report.append([area, round(mean_absolute_error(salary['pred'][salary['true_prof_area'] == area], salary['test'][salary['true_prof_area'] == area]), 1)])

# raw_desc_tfidf_RF_specialization

In [21]:
print(metrics.classification_report(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                     precision    recall  f1-score   support

 FMCG, Товары народного потребления       0.83      0.52      0.64       393
                      Автоперевозки       0.63      0.54      0.59       563
                          Бухгалтер       0.74      0.53      0.61       527
                        Ввод данных       0.86      0.24      0.38       172
                           Водитель       0.68      0.46      0.55       411
                            Грузчик       0.62      0.58      0.60       433
                   Делопроизводство       0.88      0.12      0.21       250
                        Дистрибуция       0.86      0.35      0.50        51
                             Другое       0.59      0.94      0.73      9335
                            Инженер       0.96      0.15      0.26       156
                 Кассир, Инкассатор       0.78      0.16      0.27        43
                          Кладовщик       0.49      0.28      0.36       23

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
print(tabulate(regression_report))

-----------------------------------  --------
specialization                       MAE
Управление проектами                 56776.8
Разнорабочий                         32257.7
Другое                               77117.7
Розничная торговля                   51057.5
Машиностроение                       52070.0
Официант, Бармен                     56417.2
Бухгалтер                            117541.7
Начальный уровень, Мало опыта        49917.9
Кладовщик                            34708.6
Прямые продажи                       56154.5
Водитель                             53737.3
Наладчик                             40156.0
Сотрудник call-центра                63329.1
Грузчик                              36627.7
Повар                                43641.2
Пищевая промышленность               91121.4
Делопроизводство                     43857.7
Строительство                        43635.6
Управляющий офисом (Оffice manager)  98501.4
Привлечение клиентов                 38578.1
Секретарь   

# raw_desc_tfidf_RF_prof_area

In [35]:
print(metrics.classification_report(y_test, y_pred))

                                                       precision    recall  f1-score   support

                                 Автомобильный бизнес       0.81      0.46      0.58       535
                            Административный персонал       0.56      0.54      0.55      1389
                            Банки, инвестиции, лизинг       0.89      0.81      0.85       767
                                         Безопасность       0.85      0.60      0.70       420
Бухгалтерия, управленческий учет, финансы предприятия       0.67      0.75      0.71       798
                                    Высший менеджмент       0.79      0.19      0.31       137
   Государственная служба, некоммерческие организации       0.86      0.53      0.65       123
                                         Добыча сырья       0.88      0.32      0.47       289
                                    Домашний персонал       0.79      0.12      0.20        94
                                              Зак

In [34]:
print(tabulate(regression_report))

-----------------------------------------------------  --------
prof_area                                              MAE
Туризм, гостиницы, рестораны                           87086.5
Медицина, фармацевтика                                 36096.9
Строительство, недвижимость                            50648.7
Транспорт, логистика                                   57190.8
Добыча сырья                                           51888.0
Продажи                                                79396.6
Банки, инвестиции, лизинг                              53130.2
Административный персонал                              89302.0
Информационные технологии, интернет, телеком           164694.2
Маркетинг, реклама, PR                                 151777.6
Наука, образование                                     145604.7
Рабочий персонал                                       36236.7
Бухгалтерия, управленческий учет, финансы предприятия  157228.3
Производство, сельское хозяйство                      

# lem_desc_tfidf_RF_prof_area

In [46]:
print(metrics.classification_report(y_test, y_pred))

                                                       precision    recall  f1-score   support

                                 Автомобильный бизнес       0.83      0.44      0.57       535
                            Административный персонал       0.56      0.53      0.55      1389
                            Банки, инвестиции, лизинг       0.89      0.81      0.85       767
                                         Безопасность       0.88      0.63      0.74       420
Бухгалтерия, управленческий учет, финансы предприятия       0.71      0.78      0.74       798
                                    Высший менеджмент       0.79      0.19      0.31       137
   Государственная служба, некоммерческие организации       0.90      0.52      0.66       123
                                         Добыча сырья       0.84      0.36      0.50       289
                                    Домашний персонал       0.80      0.17      0.28        94
                                              Зак

In [47]:
print(tabulate(regression_report))

-----------------------------------------------------  --------
prof_area                                              MAE
Туризм, гостиницы, рестораны                           87093.5
Медицина, фармацевтика                                 35774.5
Строительство, недвижимость                            50501.1
Транспорт, логистика                                   57046.7
Добыча сырья                                           52650.6
Продажи                                                79368.1
Банки, инвестиции, лизинг                              52999.4
Административный персонал                              89104.8
Информационные технологии, интернет, телеком           165731.4
Маркетинг, реклама, PR                                 152987.3
Наука, образование                                     143059.3
Рабочий персонал                                       36277.7
Бухгалтерия, управленческий учет, финансы предприятия  158540.6
Производство, сельское хозяйство                      

# lem_desc_tfidf_RF_specialization

In [19]:
print(metrics.classification_report(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                     precision    recall  f1-score   support

 FMCG, Товары народного потребления       0.82      0.50      0.62       393
                      Автоперевозки       0.61      0.53      0.57       563
                          Бухгалтер       0.73      0.57      0.64       527
                        Ввод данных       0.84      0.25      0.39       172
                           Водитель       0.66      0.42      0.52       411
                            Грузчик       0.64      0.60      0.62       433
                   Делопроизводство       0.90      0.11      0.19       250
                        Дистрибуция       0.82      0.35      0.49        51
                             Другое       0.60      0.94      0.73      9335
                            Инженер       0.92      0.15      0.25       156
                 Кассир, Инкассатор       0.80      0.19      0.30        43
                          Кладовщик       0.51      0.25      0.33       23

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
print(tabulate(regression_report))

-----------------------------------  --------
specialization                       MAE
Управление проектами                 56776.8
Разнорабочий                         32198.7
Другое                               77140.4
Розничная торговля                   50975.8
Машиностроение                       51776.3
Официант, Бармен                     58093.7
Бухгалтер                            118355.2
Начальный уровень, Мало опыта        49896.4
Кладовщик                            34870.8
Прямые продажи                       56929.0
Водитель                             53100.8
Наладчик                             40156.0
Сотрудник call-центра                63329.1
Грузчик                              36116.9
Повар                                41373.7
Пищевая промышленность               89928.5
Делопроизводство                     44014.8
Строительство                        43357.0
Управляющий офисом (Оffice manager)  97930.8
Привлечение клиентов                 38570.2
Секретарь   